<a href="https://colab.research.google.com/github/41monster/ML_Spring_2023/blob/main/Example_2_MLCFD_PINN_open.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Input, Model
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint
import random
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# preprocessing of training dataset
# CFD data(excel) load
filename = '/content/drive/My Drive/Colab Notebooks/Dataset_example_1.00_1.02sec_3input.xlsx'
pre_dataset = pd.read_excel(filename)

# train_stats for normalization
train_stats = pre_dataset.describe()
train_stats.pop("p")
train_stats.pop("ux")
train_stats.pop("uy")
train_stats = train_stats.transpose()

# train label
dataset = pre_dataset
train_labels_p = dataset.pop("p")
train_labels_u = dataset.pop("ux")
train_labels_v = dataset.pop("uy")

# concat train labels (for subclass model)
train_labels_p_re = tf.reshape(train_labels_p, shape=[-1,1])
train_labels_u_re = tf.reshape(train_labels_u, shape=[-1,1])
train_labels_v_re = tf.reshape(train_labels_v, shape=[-1,1])
train_labels = np.concatenate((train_labels_p_re, train_labels_u_re,train_labels_v_re),axis=1)

# normalization
def norm(x):
    return (x-train_stats['mean'])/train_stats['std']
normed_train_data = norm(dataset)
normed_train_data_re= tf.reshape(normed_train_data, shape=[-1,15])
dataset_re = tf.reshape(dataset, shape=[-1,15])

# input data (normed, raw)
#x = tf.concat([normed_train_data_re, dataset_re[:,0:1], dataset_re[:,5:6], dataset_re[:,10:11]],axis=1)
x_sum = np.concatenate((normed_train_data_re, dataset_re[:,0:1], dataset_re[:,5:6], dataset_re[:,10:11]),axis=1)
x_sum_train = x_sum[0:18000,:] # sometimes dataset size can be bigger than 18000
# output data
y_sum = train_labels
y_sum_train = train_labels[0:18000,:]

from sklearn.model_selection import train_test_split
x, x_val, y, y_val = train_test_split(x_sum_train,y_sum_train,test_size=0.2, random_state = 1004)
x_pinn = x_sum[0:18000,:]
y_pinn = y_sum[0:18000,:]

In [ ]:
# build 6 individual network models
# network architecture was determined in previous optimization study
net_p = keras.Sequential(
    [
        layers.Dense(64, activation='relu', input_shape=(15,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)   
    ],
    name="net_p",
)

net_u = keras.Sequential(
    [
        layers.Dense(64, activation='relu', input_shape=(15,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)   
    ],
    name="net_u",
)

net_v = keras.Sequential(
    [
        layers.Dense(64, activation='relu', input_shape=(15,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)   
    ],
    name="net_v",
)


In [ ]:
# loss_tracker for monotoring during training
loss_tracker = keras.metrics.Mean(name="loss")
loss_tracker_2 = keras.metrics.Mean(name="loss")
loss_tracker_3 = keras.metrics.Mean(name="loss")
mae_metric = keras.metrics.MeanAbsoluteError(name="mae")

# PINN algorithm
class FVMN_PINNs(keras.Model):
    def __init__(self, net_p, net_u, net_v, Rs, x_val=x_val, y_val=y_val, x_pinn=x_pinn, y_pinn=y_pinn):
        super(FVMN_PINNs, self).__init__()          
        self.net_p = net_p
        self.net_u = net_u
        self.net_v = net_v
        self.Rs = Rs
        self.x_val = x_val
        self.y_val = y_val
        self.x_pinn = x_pinn
        self.y_pinn = y_pinn
    
    def call(self,x):
        x = self.net_p(x)
        x = self.net_u(x)
        x = self.net_v(x)
 
    def train_step(self, data):
        x_pre, y = data
        # the order of the variables depends on the dataset.
        x = x_pre[:,0:15]
        x_ex = x_pre[:,15:18]
        x_ex = tf.cast(x_ex, dtype='float32')
        
        y_p = y[:,0:1]
        y_u = y[:,1:2] 
        y_v = y[:,2:3]
         
        with tf.GradientTape(persistent=True) as tape:
            #multi grads calculation: persistent=True
            y_pred_p = self.net_p(x)
            y_pred_u = self.net_u(x)  # Forward pass
            y_pred_v = self.net_v(x)
            
            # Calculate loss in batches.
            loss_p_mse = tf.reduce_mean(tf.square(y_p-y_pred_p), axis = 0)
            loss_u_mse = tf.reduce_mean(tf.square(y_u-y_pred_u), axis = 0)
            loss_v_mse = tf.reduce_mean(tf.square(y_v-y_pred_v), axis = 0)
                       
            # loss for training
            loss_p = loss_p_mse
            loss_u = loss_u_mse
            loss_v = loss_v_mse
            
            loss_tot = loss_p + loss_u + loss_v
        
            # conservation loss 
            x_pinn_pre = self.x_pinn
            y_pinn = self.y_pinn           
            x_pinn = x_pinn_pre[:,0:15]
            x_pinn_ex = x_pinn_pre[:,15:18]
        
            #multi grads calculation: persistent=True
            y_pred_pinn_p = self.net_p(x_pinn)
            y_pred_pinn_u = self.net_u(x_pinn)
            y_pred_pinn_v = self.net_v(x_pinn)
            
            # Residual error calculation
            y_pred_pinn_p_ex = y_pred_pinn_p + x_pinn_ex[:,0:1]
            y_pred_pinn_u_ex = y_pred_pinn_u + x_pinn_ex[:,1:2]
            y_pred_pinn_v_ex = y_pred_pinn_v + x_pinn_ex[:,2:3]
        
            y_pred_pinn_p_tf = tf.reshape(y_pred_pinn_p_ex, [180,100]) #180
            y_pred_pinn_u_tf = tf.reshape(y_pred_pinn_u_ex, [180,100])
            y_pred_pinn_v_tf = tf.reshape(y_pred_pinn_v_ex, [180,100])
            
            # continuity residual error (incompressible flow)
            y_pred_pinn_u_diff = y_pred_pinn_u_tf[2:180,1:-1] - y_pred_pinn_u_tf[0:178,1:-1] 
            y_pred_pinn_v_diff = y_pred_pinn_v_tf[1:179,2:] - y_pred_pinn_v_tf[1:179,:-2]
            
            # momentum balance residual error (incompressible flow)
            y_mom_1_tf = tf.reshape(y_pred_pinn_u, [180,100])
            y_mom_2_tf = tf.reshape(y_pred_pinn_v, [180,100])
        
            y_mom_1 = y_mom_1_tf[1:179,1:-1]
            y_mom_2 = y_mom_2_tf[1:179,1:-1]
            y_mom_3_1 = y_pred_pinn_u_tf[1:179,1:-1]
            y_mom_3_2 = y_pred_pinn_u_tf[2:180,1:-1] - y_pred_pinn_u_tf[0:178,1:-1] + y_pred_pinn_v_tf[2:180,1:-1] - y_pred_pinn_v_tf[0:178,1:-1]
            y_mom_3 = y_mom_3_1*y_mom_3_2
            y_mom_4_1 = y_pred_pinn_v_tf[1:179,1:-1]
            y_mom_4_2 = y_pred_pinn_v_tf[1:179,2:] - y_pred_pinn_v_tf[1:179,:-2] + y_pred_pinn_u_tf[1:179,2:] - y_pred_pinn_u_tf[1:179,:-2]
            y_mom_4 = y_mom_4_1*y_mom_4_2
            y_mom_5_1 = y_pred_pinn_u_tf[2:180,1:-1]-2*y_pred_pinn_u_tf[1:179,1:-1]+y_pred_pinn_u_tf[0:178,1:-1]+y_pred_pinn_v_tf[2:180,1:-1]-2*y_pred_pinn_v_tf[1:179,1:-1]+y_pred_pinn_v_tf[0:178,1:-1]
            y_mom_5_2 = y_pred_pinn_u_tf[1:179,2:]-2*y_pred_pinn_u_tf[1:179,1:-1]+y_pred_pinn_u_tf[1:179,:-2]+ y_pred_pinn_v_tf[1:179,2:]-2*y_pred_pinn_v_tf[1:179,1:-1]+y_pred_pinn_v_tf[1:179,:-2]
            y_mom_5 = 0.01*(y_mom_5_1+y_mom_5_2)
            y_mom_6 = y_pred_pinn_p_tf[2:180,1:-1] - y_pred_pinn_p_tf[0:178,1:-1] + y_pred_pinn_p_tf[1:179,2:] - y_pred_pinn_p_tf[1:179,:-2]
            
            y_mom_tot_1 = y_mom_1 + y_mom_2 + y_mom_3/2 + y_mom_4/2 - y_mom_5*100 + y_mom_6/2
        
            # due to uniform mesh
            Rs_c_1 = y_pred_pinn_u_diff + y_pred_pinn_v_diff
            Rs_c = Rs_c_1
            Rs_m = y_mom_tot_1
    
            Rs_tot_c = tf.reduce_mean(tf.square(Rs_c), axis = 0)
            Rs_tot_m = tf.reduce_mean(tf.square(Rs_m), axis = 0)
            Rs_tot = Rs_tot_c*0.1 + Rs_tot_m #0.1은 Rs_c와의 scale 보정
            
            loss_p_pinn = loss_p+ Rs_tot*0.1
            loss_u_pinn = loss_u+ Rs_tot*0.1
            loss_v_pinn = loss_v+ Rs_tot*0.1
            
            loss_tot_pinn = loss_p_pinn + loss_u_pinn + loss_v_pinn
        
        # Compute gradients
        trainable_vars_p = self.net_p.trainable_variables
        trainable_vars_u = self.net_u.trainable_variables
        trainable_vars_v = self.net_v.trainable_variables 
        
        gradients_p = tape.gradient(loss_p_pinn, trainable_vars_p)
        gradients_u = tape.gradient(loss_u_pinn, trainable_vars_u)
        gradients_v = tape.gradient(loss_v_pinn, trainable_vars_v)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients_p, trainable_vars_p))
        self.optimizer.apply_gradients(zip(gradients_u, trainable_vars_u))
        self.optimizer.apply_gradients(zip(gradients_v, trainable_vars_v))
        
        #validation loss
        x_val = self.x_val
        y_val = self.y_val 

        x_val_2 = x_val[:,0:15]      
        y_val_p = y_val[:,0:1]
        y_val_u = y_val[:,1:2] 
        y_val_v = y_val[:,2:3]
        
        y_val_pred_p = self.net_p(x_val_2)
        y_val_pred_u = self.net_u(x_val_2)  # Forward pass
        y_val_pred_v = self.net_v(x_val_2)
    
        loss_val_p_mse = tf.reduce_mean(tf.square(y_val_p-y_val_pred_p), axis = 0)
        loss_val_u_mse = tf.reduce_mean(tf.square(y_val_u-y_val_pred_u), axis = 0)
        loss_val_v_mse = tf.reduce_mean(tf.square(y_val_v-y_val_pred_v), axis = 0)
        
        val_loss = loss_val_p_mse + loss_val_u_mse + loss_val_v_mse
    
        #validation loss = validation loss + conservation error
        #Rs = self.Rs
        #val_loss = val_loss_pre + Rs_tot*Rs
    
        # loss, mse for epochs monitoring
        #loss_tracker.update_state(loss_tot)
        loss_tracker.update_state(val_loss) #Rs_tot
        loss_tracker_2.update_state(Rs_tot)
        loss_tracker_3.update_state(loss_tot)

        #mae_metric.update_state(y_u, y_pred_u)            
        #return {"loss": loss_tracker.result(), "mae": mae_metric.result()}          
        return {"val_loss": loss_tracker.result(), "Rs_tot": loss_tracker_2.result(), "loss_tot": loss_tracker_3.result()}

    @property
    #1epoch 1 initialization: https://stackoverflow.com/questions/57248723/why-there-is-sudden-drop-in-loss-after-every-epoch
    def metrics(self):
        return [loss_tracker, loss_tracker_2, loss_tracker_3] # `reset_states()` can be called automatically at the start of each epoch

import os
checkpoint_path =  './MLCFD-PINN_test'

checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                   save_weights_only=True, save_best_only=True, monitor = 'val_loss',
                                   verbose=1)
    
# model fit
model = FVMN_PINNs(net_p=net_p, net_u=net_u,net_v=net_v, Rs =0.001, x_val=x_val, y_val=y_val, x_pinn = x_pinn,y_pinn = y_pinn )

model.compile(optimizer="adam")
history = model.fit(x, y,batch_size=100, epochs=100, callbacks=[cp_callback]) # no validation data arguments